In [1]:
import tensorflow as tf

!pip install -q keras
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

1.12.0


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
DRIVE_PATH = 'gdrive/My Drive/Colab Notebooks/''

cabspotting = pd.read_pickle(DRIVE_PATH + 'cabspotting.pkl')
cabspotting.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


latitude  longitude  occupied
taxi   time                                              
abboip 2008-05-17 14:12:10  37.75153 -122.39447         0
       2008-05-17 14:13:34  37.75149 -122.39447         0
       2008-05-17 14:14:34  37.75149 -122.39447         0
       2008-05-17 14:15:35  37.75149 -122.39446         0
       2008-05-17 14:41:43  37.75144 -122.39449         0

In [3]:
cabspotting.reset_index(inplace=True)
cabspotting.head()

,taxi,time,latitude,longitude,occupied
0,abboip,2008-05-17 14:12:10,37.75153,-122.39447,0
1,abboip,2008-05-17 14:13:34,37.75149,-122.39447,0
2,abboip,2008-05-17 14:14:34,37.75149,-122.39447,0
3,abboip,2008-05-17 14:15:35,37.75149,-122.39446,0
4,abboip,2008-05-17 14:41:43,37.75144,-122.39449,0


In [4]:
# Some statistics on the lengths of the mov data per taxi
cabspotting.groupby('taxi').size().describe()

count      536.000000
mean     20932.751866
std       6204.004840
min         59.000000
25%      18721.250000
50%      22813.500000
75%      25037.750000
max      49367.000000
dtype: float64

In [5]:
import datetime as dt

cabspotting['rebased_time'] = cabspotting['time'] - cabspotting['time'].min()
cabspotting['rebased_time'] = cabspotting['rebased_time'].dt.total_seconds()
cabspotting.head()

,taxi,time,latitude,longitude,occupied,rebased_time
0,abboip,2008-05-17 14:12:10,37.75153,-122.39447,0,15126.0
1,abboip,2008-05-17 14:13:34,37.75149,-122.39447,0,15210.0
2,abboip,2008-05-17 14:14:34,37.75149,-122.39447,0,15270.0
3,abboip,2008-05-17 14:15:35,37.75149,-122.39446,0,15331.0
4,abboip,2008-05-17 14:41:43,37.75144,-122.39449,0,16899.0


In [6]:
cabspotting.drop(['time', 'occupied'], axis=1, inplace=True)
cabspotting.head()

,taxi,latitude,longitude,rebased_time
0,abboip,37.75153,-122.39447,15126.0
1,abboip,37.75149,-122.39447,15210.0
2,abboip,37.75149,-122.39447,15270.0
3,abboip,37.75149,-122.39446,15331.0
4,abboip,37.75144,-122.39449,16899.0


In [7]:
def extract_sequence(df):
    df.drop('taxi', axis=1, inplace=True)
    df.sort_values(by='rebased_time', inplace=True)
    return df.values

cabspotting = cabspotting.groupby('taxi').apply(extract_sequence)
cabspotting.head()

taxi
abboip      [[37.751529999999995, -122.39447, 15126.000000...
abcoij      [[37.80346, -122.41466000000001, 47486.0], [37...
abdremlu    [[37.75521, -122.39093000000001, 11949.0], [37...
abgibo      [[37.7733, -122.4374, 16.0], [37.77791, -122.4...
abjoolaw    [[37.75159, -122.39747, 13856.0], [37.74949, -...
dtype: object

In [11]:
cabspotting['abboip']

array([[ 3.7751530e+01, -1.2239447e+02,  1.5126000e+04],
       [ 3.7751490e+01, -1.2239447e+02,  1.5210000e+04],
       [ 3.7751490e+01, -1.2239447e+02,  1.5270000e+04],
       ...,
       [ 3.7751990e+01, -1.2239460e+02,  2.0661360e+06],
       [ 3.7751360e+01, -1.2239527e+02,  2.0662550e+06],
       [ 3.7751340e+01, -1.2239488e+02,  2.0662830e+06]])

In [25]:
data = None
for _, item in cabspotting.iteritems():
    if data is None:
        data = item
    else:
        data = np.concatenate((data, item), axis=0)

data.shape

(11219955, 3)

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, sequence_length=20, batch_size=32):
        'Initialization'
        data_length = data.shape[0]
        n_batches = (data_length - 1) // (batch_size * sequence_length)
        rounded_data_length = n_batches * batch_size * sequence_length
        
        X_data = np.reshape(data[0:rounded_data_length], [batch_size, n_batches * sequence_length, 3])
        y_data = np.reshape(data[1:rounded_data_length + 1], [batch_size, n_batches * sequence_length, 3])
        
        self.n_batches = n_batches
        self.sequence_length = sequence_length
        self.X_data = X_data
        self.y_data = y_data
        
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.n_batches
    
    def __getitem__(self, index):
        'Generate one batch of data'        
        X = self.X_data[:, index * self.sequence_length:(index + 1) * self.sequence_length]
        y = self.y_data[:, index * self.sequence_length:(index + 1) * self.sequence_length]
        
        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.X_data = np.roll(self.X_data, -1, axis=0)
        self.y_data = np.roll(self.y_data, -1, axis=0)

generator = DataGenerator(data)

In [44]:
model = keras.Sequential([
    keras.layers.GRU(512, return_sequences=True, input_shape=(20, 3)),
    keras.layers.TimeDistributed(keras.layers.Dense(3, activation='relu'))
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 20, 512)           792576    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 3)             1539      
Total params: 794,115
Trainable params: 794,115
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Callback for saving checkpoints
filepath = DRIVE_PATH + 'one-gru-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [46]:
model.fit_generator(generator, epochs=3, callbacks=[checkpoint])

Epoch 1/40
 3375/17531 [====>.........................] - ETA: 13:40 - loss: 1256.7301 - mean_squared_error: 459386655769.7896

KeyboardInterrupt: ignored